In [1]:
-- connection: host='localhost' dbname='Photographies' user='postgres' password='postgres'

# Point de vue utilisateur:

##  La photo dans laquelle figure une personne.(fonction)

In [2]:
/*====================================================
 * La photo dans laquelle figure une personne.(fonction)
 *====================================================*/
DROP FUNCTION IF EXISTS chercher_photo_par_personne;
CREATE OR REPLACE FUNCTION chercher_photo_par_personne(str varchar)
RETURNS TABLE(Article integer, NomOeuvre varchar) AS $$
    BEGIN
        RETURN QUERY(
            SELECT PhotoArticle, I.NomOeuvre FROM
            Document D JOIN IndexPersonne I on D.idOeuvre=I.idOeuvre
            WHERE I.NomOeuvre ILIKE '%' || str || '%'
        );
    END; 
$$ language plpgsql;

SELECT * FROM chercher_photo_par_personne('Becquerel') LIMIT 10;

10 row(s) returned.


article,nomoeuvre
10923,"BECQUEREL, Antoine-César (physicien)"
13889,"BECQUEREL, Antoine-César (physicien)"
13890,"BECQUEREL, Antoine-César (physicien)"
16626,"BECQUEREL, Antoine-César (physicien)"
16633,"BECQUEREL, Antoine-César (physicien)"
17825,"BECQUEREL, Antoine-César (physicien)"
17830,"BECQUEREL, Antoine-César (physicien)"
17831,"BECQUEREL, Antoine-César (physicien)"
17832,"BECQUEREL, Antoine-César (physicien)"
17840,BECQUEREL (famille)


## Les photos qui présentent un sujet.(fonction)

In [3]:
/*====================================================
 * Les photos qui présentent un sujet.(fonction)
 *====================================================*/
DROP FUNCTION IF EXISTS chercher_photo_par_sujet;
CREATE OR REPLACE FUNCTION chercher_photo_par_sujet(str varchar)
RETURNS TABLE(PhotoArticle integer, NomOeuvre varchar) AS $$
    BEGIN
        RETURN QUERY(
            SELECT D.PhotoArticle, S.DescSujet FROM
            Document D JOIN Sujet S on D.idSujet = S.idSujet
            WHERE S.DescSujet ILIKE '%' || str || '%'
        );
    END;  
$$ language plpgsql;

SELECT * FROM chercher_photo_par_sujet('église') LIMIT 10;

10 row(s) returned.


photoarticle,nomoeuvre
2,église Notre-Dame-de-Recouvrance
2,église Notre-Dame-de-Recouvrance
2,église Notre-Dame-de-Recouvrance
2,église Notre-Dame-de-Recouvrance
7,église
7,église
8,église
9,église
9,église
9,église


## Afficher que les photos en couleur (view)

In [4]:
/*====================================================
 * Afficher que les photos en couleur (view)
 *====================================================*/
DROP VIEW IF EXISTS photo_en_couleur;
CREATE OR REPLACE VIEW photo_en_couleur AS
    SELECT Article, Remarques, NbrCli, DescDet, idSerie  
    FROM Photo P JOIN Document D ON P.Article = D.PhotoArticle
    WHERE d.C_G = 'CLR';

SELECT * FROM photo_en_couleur LIMIT 10;

10 row(s) returned.


article,remarques,nbrcli,descdet,idserie
803,même vue expo différente,2,Tableau sur retable,1
871,même vue expo différente,1,Tribune et orgue,1
871,même vue expo différente,1,Tribune et orgue,1
926,même vue expo différente,2,"Trésors de la Cathédrale, petit reliquaire à roues",1
927,même vue expo différente,1,Crosse de Manassés de Seignelay,1
929,même vue expo différente,2,Médaillon de gants de Raoul Grosparmi,1
930,même vue expo différente,1,Crosse de Robert de Courtenay,1
1001,même vue expo différente,2,Chaire,1
1001,même vue expo différente,2,Chaire,1
1155,inversible : même vue expo différente,1,Retable,1


## Afficher que les photos en noir et blanc (view)

In [5]:
/*====================================================
 * Afficher que les photos en noir et blanc (view)
 *====================================================*/
DROP VIEW IF EXISTS photo_en_noirblanc;
CREATE OR REPLACE VIEW photo_en_noirblanc AS
    SELECT DISTINCT(Article), Remarques, NbrCli, DescDet, idSerie  
    FROM Photo P JOIN Document D ON P.Article = D.PhotoArticle
    WHERE d.C_G = 'GSC';

SELECT * FROM photo_en_noirblanc LIMIT 10;

10 row(s) returned.


article,remarques,nbrcli,descdet,idserie
715,,1,Rez-de-chaussée,1
7514,,1,,1
13834,,1,,1
4228,,1,,1
21884,,1,,1
13989,,1,,1
12276,,1,,1
25598,même vue expo différente,1,,1
24900,,1,,1
13590,,1,,1


## Les photos qui ont un fichier numérique.(view)

In [6]:
/*====================================================
 * Les photos qui ont un fichier numérique.(view)
 *====================================================*/ 
CREATE OR REPLACE VIEW view_photo_numeric AS
    SELECT DISTINCT(Article), Remarques, NbrCli, DescDet, idSerie  
    FROM Photo P JOIN Document D ON P.Article = D.PhotoArticle
    WHERE d.FicNum IS NOT NULL;

SELECT * FROM view_photo_numeric LIMIT 10;

10 row(s) returned.


article,remarques,nbrcli,descdet,idserie
7514,,1,,1
4228,,1,,1
34353,,1,,1
23853,,1,,1
24795,,1,,1
5839,,1,,1
11918,,1,,1
9882,,1,,1
25236,,1,,1
12226,,1,,1


## Les photos qui correspondent au fichier donné.(fonction)

In [7]:
/*====================================================
 * Les photos qui correspondent au fichier donné.(fonction)
 *====================================================*/ 
DROP FUNCTION IF EXISTS rechercher_cliche_photo;
CREATE OR REPLACE FUNCTION rechercher_cliche_photo(varchar)
    RETURNS TABLE(article integer, ficnum varchar, nbrcli integer, N_V varchar, C_G varchar, idserie integer)     
    LANGUAGE 'plpgsql'
AS $BODY$
    BEGIN
        RETURN QUERY(
            select P.Article,D.ficnum,P.NbrCli,D.N_V,D.C_G,P.idSerie from Photo P
            JOIN document D ON P.article = D.photoarticle
            where  D.ficnum= $1
        );
    END; 
$BODY$;

SELECT * FROM rechercher_cliche_photo('FRAD045_CLVUE000001_H');

2 row(s) returned.


article,ficnum,nbrcli,n_v,c_g,idserie
1,FRAD045_CLVUE000001_H,3,NEG,GSC,1
1,FRAD045_CLVUE000001_H,3,NEG,GSC,1


## Chercher un certain type d’oeuvre.

In [8]:
/*====================================================
 * Chercher un certain type d’oeuvre.
 *====================================================*/ 
DROP FUNCTION IF EXISTS rechercher_type_oeuvre;
CREATE OR REPLACE FUNCTION rechercher_type_oeuvre(varchar)
RETURNS TABLE(article integer, ficnum varchar, nbrcli integer, N_V varchar, C_G varchar, idserie integer)    
AS $$
    BEGIN
        RETURN QUERY( 
            select DISTINCT(P.Article), D.ficnum,P.NbrCli,D.N_V,D.C_G,P.idSerie from Photo P
            join Document D on D.PhotoArticle=P.Article
            join IndexPersonne I on D.idoeuvre=I.idoeuvre
            join TypeOeuvre T on I.typeOeuvre=T.idType
            where T.NomType = $1
        );
    END;
$$ LANGUAGE plpgsql;

select * from rechercher_type_oeuvre('statue') LIMIT 10;

10 row(s) returned.


article,ficnum,nbrcli,n_v,c_g,idserie
134,,1,NEG,GSC,1
135,,1,NEG,GSC,1
397,,1,NEG,GSC,1
398,,1,NEG,GSC,1
915,,1,NEG,GSC,1
916,,1,NEG,GSC,1
920,,1,NEG,GSC,1
921,,2,NEG,GSC,1
922,,2,NEG,GSC,1
923,,1,NEG,GSC,1


## Rechercher les photos qui appartiennent à une série.

In [9]:
/*====================================================
 * Rechercher les photos qui appartiennent à une série.
 *====================================================*/ 
DROP FUNCTION IF EXISTS rechercher_serie_photo;
CREATE OR REPLACE FUNCTION rechercher_serie_photo(varchar)
RETURNS TABLE(article integer, ficnum varchar, nbrcli integer, N_V varchar, C_G varchar, idserie integer)    
AS $$
    BEGIN
        RETURN QUERY(
            select DISTINCT(P.Article), D.ficnum, P.NbrCli, D.N_V, D.C_G, P.idSerie from Photo P
            join Document D on P.Article=D.PhotoArticle
            join Serie S on S.IdSerie=P.IdSerie
            where S.nomSerie = $1
        );
        END;
$$ LANGUAGE plpgsql;

select * from rechercher_serie_photo('CLVUE') LIMIT 10;

10 row(s) returned.


article,ficnum,nbrcli,n_v,c_g,idserie
1,FRAD045_CLVUE000001_H,3,NEG,GSC,1
2,,1,NEG,GSC,1
3,,1,NEG,GSC,1
4,,4,NEG,GSC,1
5,,1,NEG,GSC,1
6,,1,NEG,GSC,1
7,,1,NEG,GSC,1
8,,2,NEG,GSC,1
9,,1,NEG,GSC,1
10,,1,NEG,GSC,1


## Les photos correspondant à un index iconographique précis

In [10]:
/*====================================================
 * Les photos correspondant à un index iconographique précis
 *====================================================*/
CREATE OR REPLACE FUNCTION index_photo(varchar)
RETURNS TABLE(article integer, ficnum varchar, nbrcli integer, N_V varchar, C_G varchar, idserie integer)    
  AS $$
    BEGIN
RETURN QUERY

select P.Article,D.ficnum,P.NbrCli,D.N_V,D.C_G,P.idSerie from Photo P
join Document D on P.Article=D.PhotoArticle
join IndexIconographique I on I.IdIco = D.idico
where I.idx_ico = $1;
    END;
$$ LANGUAGE plpgsql;

select * from index_photo('paysage urbain') LIMIT 10;

10 row(s) returned.


article,ficnum,nbrcli,n_v,c_g,idserie
1,FRAD045_CLVUE000001_H,3,NEG,GSC,1
2,,1,NEG,GSC,1
16,,1,NEG,GSC,1
21,,1,NEG,GSC,1
23,,1,NEG,GSC,1
30,,1,NEG,GSC,1
54,,4,NEG,GSC,1
272,,1,NEG,GSC,1
273,,1,NEG,GSC,1
326,,1,NEG,GSC,1


## Le nombre de photos dont on ne connait pas la date

In [11]:
/*====================================================
 * Le nombre de photos dont on ne connait pas la date
 *====================================================*/
DROP FUNCTION IF EXISTS date_photo CASCADE;
CREATE OR REPLACE FUNCTION date_photo()
RETURNS TABLE(article integer, ficnum varchar, nbrcli integer, N_V varchar, C_G varchar, idserie integer)    
  AS $$
    BEGIN
RETURN QUERY
select distinct(P.Article),D.ficnum,P.NbrCli,D.N_V,D.C_G,P.idSerie from Photo P
join Document D on P.Article=D.PhotoArticle
where  D.iddate is Null;
END;
$$ LANGUAGE plpgsql;

select COUNT(distinct article) from date_photo();

1 row(s) returned.


count
394


## Le nombre de photos dont on ne connaît pas la ville

In [12]:
/*====================================================
 * Le nombre de photos dont on ne connait pas la ville
 *====================================================*/
drop function if exists ville_photo();
CREATE OR REPLACE FUNCTION ville_photo()
RETURNS TABLE(article integer, ficnum varchar, nbrcli integer, N_V varchar, C_G varchar, idserie integer) AS $$
    BEGIN
        RETURN QUERY
            select P.Article,D.ficnum,P.NbrCli,D.N_V,D.C_G,P.idSerie from Photo P
            join Document D on P.Article=D.PhotoArticle
            where D.idville is Null;
        END;
$$ LANGUAGE plpgsql;

select count(distinct article) from ville_photo();

1 row(s) returned.


count
321


## Le nombre moyen de photo par ville

In [13]:
/*====================================================
 * Le nombre moyen de photo par ville
 *====================================================*/
DROP FUNCTION IF EXISTS func_avg_cliche_ville CASCADE;
CREATE OR REPLACE FUNCTION func_avg_photo_ville()
RETURNS float AS $$
    BEGIN
        RETURN (
            WITH subquery AS (SELECT NomVille, COUNT(DISTINCT PhotoArticle) as nbPhoto
            FROM Document d, Ville v
            WHERE d.idVille = v.idVille
            GROUP BY NomVille)
            SELECT AVG(nbPhoto) from subquery
        );
    END;
$$ language plpgsql;

SELECT func_avg_photo_ville();

1 row(s) returned.


func_avg_photo_ville
83.0296


## Le nombre moyen de cliché par ville

In [14]:
/*====================================================
 * Le nombre moyen de cliché par ville
 *====================================================*/
drop function if exists func_avg_cliche_ville() CASCADE;
CREATE OR REPLACE FUNCTION func_avg_cliche_ville()
RETURNS float AS $$
BEGIN
    RETURN(
        WITH subquery AS (
            SELECT NomVille, SUM(NbrCli::integer) as nbCliche
            FROM Photo p, Document d, Ville v
            WHERE p.Article = d.PhotoArticle and d.idVille = v.idVille
            GROUP BY NomVille
        )
        SELECT AVG(nbCliche) from subquery
    );
END;
$$ LANGUAGE plpgsql;

select func_avg_cliche_ville();

1 row(s) returned.
NOTICE:  function func_avg_cliche_ville() does not exist, skipping


func_avg_cliche_ville
360.386


In [15]:
/*====================================================
 * Les n photos les plus anciennes 
 * flag: true (plus anciens), false (plus récent)
 *====================================================*/
DROP FUNCTION IF EXISTS func_photos_anciennes_recents CASCADE;
CREATE OR REPLACE FUNCTION func_photos_anciennes_recents(n integer, flag boolean)
RETURNS TABLE(PhotoArticle integer, Discriminant integer, DateP varchar) AS $$
    BEGIN
        IF flag THEN
            RETURN QUERY(
                WITH subqueryYear AS(SELECT MIN(DateAnnee::numeric) as minYear FROM DatePhoto),
                subquery1 AS(
                    SELECT D.PhotoArticle as pArticle, D.Discriminant as pDiscriminant, DateJour, DateMois, DateAnnee
                    FROM Document D, DatePhoto Da 
                    WHERE D.idDate = Da.idDate and Da.DateAnnee::numeric = (SELECT minYear FROM subqueryYear)
                    ORDER BY DateMois, DateAnnee ASC
                )
                SELECT pArticle, pDiscriminant, array_to_string(ARRAY[DateJour, num_to_month(DateMois), DateAnnee], ' ', '')::varchar 
                FROM subquery1
                LIMIT n
            );
        ELSE
            RETURN QUERY(
                WITH subqueryYear AS(SELECT MAX(DateAnnee::numeric) as minYear FROM DatePhoto),
                subquery1 AS(
                    SELECT D.PhotoArticle as pArticle, D.Discriminant as pDiscriminant, DateJour, DateMois, DateAnnee
                    FROM Document D, DatePhoto Da 
                    WHERE D.idDate = Da.idDate and Da.DateAnnee::numeric = (SELECT minYear FROM subqueryYear)
                    ORDER BY DateMois, DateAnnee DESC
                )
                SELECT pArticle, pDiscriminant, array_to_string(ARRAY[DateJour, num_to_month(DateMois), DateAnnee], ' ', '')::varchar 
                FROM subquery1
                LIMIT n
            );
        END IF;
    END; 
$$ LANGUAGE plpgsql;

SELECT * FROM func_photos_anciennes_recents(10, true);

10 row(s) returned.


photoarticle,discriminant,datep
13,21,1961
1,2,1961
2,1,1961
2,2,1961
2,3,1961
2,4,1961
3,1,1961
3,2,1961
4,1,1961
4,2,1961


In [16]:
CREATE OR REPLACE VIEW view_statistique AS 
    SELECT func_avg_cliche_ville() as nbre_moyen_cliche ,
        func_avg_photo_ville() as nbre_moyen_photo,
        (select count(*) from date_photo()) as photos_sans_date,
        (select ARRAY[PhotoArticle::varchar, DateP] FROM func_photos_anciennes_recents(1, true)) as photo_plus_ancienne,
        (select ARRAY[PhotoArticle::varchar, DateP] FROM func_photos_anciennes_recents(1, false)) as photo_plus_recente
;

select * from view_statistique;

1 row(s) returned.


nbre_moyen_cliche,nbre_moyen_photo,photos_sans_date,photo_plus_ancienne,photo_plus_recente
360.386,83.0296,394,"['13', ' 1961']","['33124', '26 Janvier 2001']"


# Proportions et évolution
## Fonction qui permet de faire les histogrames

In [17]:
/*================================================
 * FUNCTION: histogram
 * Représenter les pourcentages en visuel
 *================================================*/
DROP FUNCTION IF EXISTS histogram(anyarray,anyelement);
CREATE OR REPLACE FUNCTION histogram(arr anyarray, freq anyelement)
RETURNS varchar AS $$
    DECLARE
        v_max integer := (SELECT MAX(a) FROM unnest(arr) a);
    BEGIN
        IF v_max = 0 THEN v_max := 1; END IF;
        RETURN repeat('█', (freq::float/(v_max) *30)::int);
    
    END;
$$ language plpgsql;

## Le pourcentage de photos pris dans chaque ville. (la ville qui a plus de photos)

In [18]:
/*================================================
 * Le pourcentage de photos pris dans chaque ville. (la ville qui a plus de photos)
 *================================================*/
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT NomVille, 
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, Ville v
        WHERE d.idVille = v.idVille
        GROUP BY NomVille
    )
SELECT NomVille, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY nbPhoto DESC
LIMIT 10;

10 row(s) returned.


nomville,nbphoto,pourcentage,bar
Orléans,6269,18.2291,██████████████████████████████
Gien,841,2.44548,████
Montargis,651,1.89299,███
Pithiviers,525,1.52661,███
Olivet,475,1.38122,██
Beaugency,434,1.26199,██
Sully-sur-Loire,427,1.24164,██
Loire(fleuve),421,1.22419,██
La-Ferté-Saint-Aubin,418,1.21547,██
Beaune-la-Rolande,396,1.1515,██


## Le pourcentage de prise de photos en fonction des années.(évolution sur une période)

In [19]:
/*================================================
 * Le pourcentage de prise de photos en fonction des années.(évolution sur une période)
 *================================================*/
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT DateAnnee::numeric as Annee, 
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, DatePhoto dp
        WHERE d.idDate = dp.idDate
        GROUP BY Annee
    )
SELECT Annee, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY nbPhoto DESC
LIMIT 10;

10 row(s) returned.


annee,nbphoto,pourcentage,bar
1975,6370,18.5228,█████████████████████████████
1996,2226,6.47281,██████████
1973,1551,4.51003,███████
1995,1469,4.27159,███████
1999,1362,3.96045,██████
2001,1329,3.8645,██████
1983,1303,3.78889,██████
1982,1262,3.66967,██████
1998,1233,3.58534,██████
1969,1129,3.28293,█████


## La proportion de photos selon leur taille. 

In [20]:
/*================================================
 * La proportion de photos selon leur taille.
 *================================================*/
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT Taille, 
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, Cliche c
        WHERE d.idCliche = c.idCliche
        GROUP BY Taille
    )
SELECT Taille, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY nbPhoto DESC
LIMIT 10;

7 row(s) returned.


taille,nbphoto,pourcentage,bar
"10x12.5 (4x5"")",22687,65.9698,██████████████████████████████
24x36,7224,21.0061,██████████
4.5x6,3013,8.76127,████
9x12,1817,5.28351,██
6x7,51,0.148299,
6x9,14,0.0407095,
6x6,9,0.0261704,


## Le pourcentage de personnes représentées ( + par ville, par année)

In [21]:
/*================================================
 * Le pourcentage de personnes représentées ( + par ville, par année)
 *================================================*/
 
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT NomOeuvre, 
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, IndexPersonne p
        WHERE d.idOeuvre = p.idOeuvre
        GROUP BY NomOeuvre
    )
SELECT NomOeuvre, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY nbPhoto DESC
LIMIT 10;

10 row(s) returned.


nomoeuvre,nbphoto,pourcentage,bar
JEANNE D'ARC,166,0.482698,██████████████
CHIRAC Jacques (président de la République),143,0.415819,████████████
"BOLLÉE (famille), voir aussi Saint-Jean-de-Braye",95,0.276243,████████
DELMAS (ébéniste décorateur),70,0.203548,██████
"HUBERT Jean (joaillier, sculpteur, héraldiste)",70,0.203548,██████
BERNEAU (pêcheur de saumons en Loire),35,0.101774,███
SOLON Bernard (taillendier),33,0.0959581,███
BOULET René (sabotier),32,0.0930503,███
AVEZARD Pierre,27,0.0785112,██
DABARD (distillateur),26,0.0756034,██


In [22]:
/*================================================
 * Le pourcentage de personnes représentées ( par ville)
 *================================================*/
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT NomOeuvre, NomVille,
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, IndexPersonne p, Ville v
        WHERE d.idOeuvre = p.idOeuvre and d.idVille = v.idVille
        GROUP BY NomOeuvre, NomVille
    )
SELECT NomOeuvre, NomVille, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY nbPhoto DESC
LIMIT 10;

10 row(s) returned.


nomoeuvre,nomville,nbphoto,pourcentage,bar
JEANNE D'ARC,Orléans,154,0.447805,█████████████
CHIRAC Jacques (président de la République),Orléans,143,0.415819,████████████
"BOLLÉE (famille), voir aussi Saint-Jean-de-Braye",Saint-Jean-de-Braye,95,0.276243,████████
DELMAS (ébéniste décorateur),Nargis,70,0.203548,██████
DELMAS (ébéniste décorateur),Chécy,70,0.203548,██████
DELMAS (ébéniste décorateur),Bellegarde,70,0.203548,██████
DELMAS (ébéniste décorateur),Châtillon-le-Roi,70,0.203548,██████
DELMAS (ébéniste décorateur),Gien,70,0.203548,██████
DELMAS (ébéniste décorateur),Mignerette,70,0.203548,██████
DELMAS (ébéniste décorateur),Montargis,70,0.203548,██████


In [23]:
/*================================================
 * Le pourcentage de personnes représentées ( par année)
 *================================================*/
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT NomOeuvre, DateAnnee::numeric,
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, IndexPersonne p, DatePhoto dp
        WHERE d.idOeuvre = p.idOeuvre and d.idDate = dp.idDate
        GROUP BY NomOeuvre, DateAnnee
    )
SELECT NomOeuvre, DateAnnee, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY NomOeuvre
LIMIT 10;

10 row(s) returned.


nomoeuvre,dateannee,nbphoto,pourcentage,bar
AVEZARD Pierre,1982,27,0.0785112,██
BEAUDIN (chaisier),1982,16,0.0465252,█
"BEAUHARNAIS, Anne de (marquise de Cepoy)",1975,1,0.00290782,
"BECQUEREL, Antoine-César (physicien)",1982,4,0.0116313,
"BECQUEREL, Antoine-César (physicien)",1975,3,0.00872347,
"BECQUEREL, Antoine-César (physicien)",1993,1,0.00290782,
"BECQUEREL, Antoine-César (physicien)",1981,2,0.00581564,
BECQUEREL (famille),1982,6,0.0174469,█
BERNEAU (pêcheur de saumons en Loire),1963,5,0.0145391,
BERNEAU (pêcheur de saumons en Loire),1962,17,0.049433,█


## Le pourcentage de type d’oeuvre représentées (+ par ville, par année)

In [24]:
/*================================================
 * Le pourcentage de type d’oeuvre représentées (+ par ville, par année)
 *================================================*/
 
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT NomType,
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, IndexPersonne p, TypeOeuvre t
        WHERE d.idOeuvre = p.idOeuvre and p.TypeOeuvre = t.idType
        GROUP BY NomType
    )
SELECT NomType, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY nbPhoto DESC
LIMIT 10;

10 row(s) returned.


nomtype,nbphoto,pourcentage,bar
,666,1.93661,█████████████████████████████
fête,143,0.415819,██████
statue,122,0.354754,█████
fonderie Bollée,95,0.276243,████
plaque commémorative,31,0.0901425,█
monument commémoratif,7,0.0203548,
tombeau,6,0.0174469,
maison,6,0.0174469,
mausolée,6,0.0174469,
foyer,5,0.0145391,


In [25]:
/*================================================
 * Le pourcentage de type d’oeuvre représentées (+ par ville)
 *================================================*/
 
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT NomType, NomVille,
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, IndexPersonne p, TypeOeuvre t, Ville v
        WHERE d.idOeuvre = p.idOeuvre and p.TypeOeuvre = t.idType and d.idVille = v.idVille
        GROUP BY NomType, NomVille
    )
SELECT NomType, NomVille, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY nbPhoto DESC
LIMIT 10;

10 row(s) returned.


nomtype,nomville,nbphoto,pourcentage,bar
,Orléans,297,0.863623,██████████████████████████
fête,Orléans,143,0.415819,████████████
fonderie Bollée,Saint-Jean-de-Braye,95,0.276243,████████
,Nargis,88,0.255888,████████
,Montargis,80,0.232626,███████
,Gien,76,0.220994,███████
,Saint-Jean-de-Braye,76,0.220994,███████
,Chécy,75,0.218087,███████
,Mignerette,70,0.203548,██████
,Châtillon-le-Roi,70,0.203548,██████


In [26]:
/*================================================
 * Le pourcentage de type d’oeuvre représentées (+ par année)
 *================================================*/
 
WITH 
    subquery AS(SELECT COUNT(Article) as nbPhoto FROM Photo),
    query AS(
        SELECT NomType, DateAnnee::numeric,
            COUNT(DISTINCT PhotoArticle) as nbPhoto, 
            COUNT(DISTINCT PhotoArticle)*100/(SELECT nbPhoto FROM subquery) ::float as pourcentage
        FROM Document d, IndexPersonne p, TypeOeuvre t, DatePhoto dp
        WHERE d.idOeuvre = p.idOeuvre and p.TypeOeuvre = t.idType and d.idDate = dp.idDate
        GROUP BY NomType, DateAnnee
    )
SELECT NomType, DateAnnee, nbPhoto, pourcentage, histogram(ARRAY(SELECT pourcentage from query), pourcentage) as bar
FROM query
ORDER BY nbPhoto DESC
LIMIT 10;

10 row(s) returned.


nomtype,dateannee,nbphoto,pourcentage,bar
,1975,211,0.61355,██████████████████
,1996,145,0.421634,█████████████
fête,1996,143,0.415819,████████████
,1982,112,0.325676,██████████
fonderie Bollée,1975,95,0.276243,████████
,1999,42,0.122129,████
,1972,32,0.0930503,███
,1994,23,0.0668799,██
,1974,23,0.0668799,██
,1962,22,0.0639721,██
